In [1]:
import pyglet
import numpy as np
import colorsys

grid_width = 100
grid_height = 100

cell_size = 8

SCREEN_WIDTH = cell_size*grid_width
SCREEN_HEIGHT = cell_size*grid_height

grid_pos = np.zeros((grid_width, grid_height), dtype=np.float32)
grid_vel = np.zeros((grid_width, grid_height), dtype=np.float32)

time = 0

win = pyglet.window.Window(SCREEN_WIDTH, SCREEN_HEIGHT, caption = "Standing waves")

def temp_to_colour(temp):
    hue = (1-temp)*0.66
    r, g, b = colorsys.hsv_to_rgb(hue, 1, 1)
    return (int(r*255), int(g*255), int(b*255))
    
batch1 = pyglet.graphics.Batch()
sprites = []
for i in range(grid_width):
    column = []
    for j in range(grid_height):
        column.append(pyglet.shapes.Rectangle(i*cell_size, j*cell_size, cell_size, cell_size, color=temp_to_colour(grid_pos[i][j]), batch=batch1))
    sprites.append(column)

dt = 0.0001

@win.event
def on_draw():
    win.clear()
    batch1.draw()

alpha = 5
frequency = 0.025

def update(dt):
    global grid_pos, grid_vel, time
    time += dt
    grid_pos[grid_width//2][grid_height//2] = np.sin(time*frequency*2*np.pi)
    old_pos = grid_pos.copy()

    for i in range(1, grid_width - 1):
        for j in range(1, grid_height -1):
            laplacian = (old_pos[i+1][j] + old_pos[i-1][j] + old_pos[i][j-1] + old_pos[i][j+1] - 4*old_pos[i][j])
            acceleration = alpha*laplacian
            grid_vel[i][j] += acceleration*dt

    for i in range(1, grid_width - 1):
        for j in range(1, grid_height - 1):
            grid_pos[i][j] += grid_vel[i][j] * dt

    for i in range(grid_width):
        for j in range(grid_height):
            val = np.clip((grid_pos[i][j] + 1) / 2.0, 0, 1)
            sprites[i][j].color = temp_to_colour(val)
            
if __name__ == "__main__":
    pyglet.clock.schedule(update)
    pyglet.app.run()